In [ ]:
# dataset: full TLE history of satellite 48445 in 2024

# Estimate the total count of satellite maneuvers, as change of semi-major axis of over 100m within 12 hours, for the satellite with id 48445, using the provided TLE history.  

In [1]:
# Q5

import numpy as np
from datetime import timedelta
from skyfield.api import load, EarthSatellite

def read_tle_file(filename):
    """Reads a TLE file and returns a list of (line1, line2) tuples."""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    if len(lines) % 2 != 0:
        raise ValueError("TLE file should contain an even number of lines.")
    return [(lines[i], lines[i+1]) for i in range(0, len(lines), 2)]

def compute_semi_major_axis(satellite):
    """Computes the semi-major axis in kilometers from the satellite's mean motion."""
    mu = 398600.4418  # Earth's gravitational parameter, km^3/s^2
    n = satellite.model.no_kozai  # Mean motion in radians per minute
    n_rad_s = n * (1 / 60)  # Convert to radians per second
    a_m = (mu / n_rad_s**2) ** (1/3)  # Semi-major axis in meters
    return a_m  

def detect_maneuvers(tle_pairs, threshold_km=0.1, t_hours=12):
    """
    Detects maneuvers based on changes in semi-major axis within a specified time window.
    
    Parameters:
    - tle_pairs: List of TLE line pairs.
    - threshold_km: Threshold for detecting maneuvers in kilometers.
    - t_hours: Time window in hours to consider for maneuver detection.
    
    Returns:
    - List of tuples containing the epoch and delta_a where maneuvers are detected.
    """
    ts = load.timescale()
    semi_major_axes = []
    epochs = []

    for line1, line2 in tle_pairs:
        satellite = EarthSatellite(line1, line2, ts=ts)
        a = compute_semi_major_axis(satellite)
        semi_major_axes.append(a)
        epochs.append(satellite.epoch.utc_datetime())

    maneuvers = []
    for i in range(1, len(semi_major_axes)):
        delta_a = abs(semi_major_axes[i] - semi_major_axes[i - 1])
        delta_t = (epochs[i] - epochs[i - 1]).total_seconds() / 3600.0  # Convert to hours
        if delta_a > threshold_km and delta_t <= t_hours:
            maneuvers.append((epochs[i], delta_a))

    return maneuvers


tle_filename = 'astro_q5/48445.tle'  # Replace with your TLE file path
threshold_km = 0.1  # Threshold for detecting maneuvers in kilometers
hour_interval = 12

try:
    tle_pairs = read_tle_file(tle_filename)
except Exception as e:
    print(f"Error reading TLE file: {e}")

maneuvers = detect_maneuvers(tle_pairs, threshold_km, hour_interval)

print(f"Total maneuvers detected: {len(maneuvers)}")
for epoch, delta_a in maneuvers:
    print(f"Maneuver detected at {epoch} with delta_a = {delta_a:.3f} km")


Total maneuvers detected: 20
Maneuver detected at 2024-02-13 12:03:37.671844+00:00 with delta_a = 0.123 km
Maneuver detected at 2024-02-14 03:59:01.910095+00:00 with delta_a = 0.146 km
Maneuver detected at 2024-04-20 04:20:52.293121+00:00 with delta_a = 0.122 km
Maneuver detected at 2024-05-11 17:51:14.271576+00:00 with delta_a = 0.152 km
Maneuver detected at 2024-05-12 19:19:54.177887+00:00 with delta_a = 0.107 km
Maneuver detected at 2024-05-25 22:36:42.852966+00:00 with delta_a = 0.113 km
Maneuver detected at 2024-08-29 11:33:38.170654+00:00 with delta_a = 0.106 km
Maneuver detected at 2024-08-31 11:19:49.926453+00:00 with delta_a = 0.105 km
Maneuver detected at 2024-09-01 03:15:14.438568+00:00 with delta_a = 0.132 km
Maneuver detected at 2024-09-02 20:39:16.721283+00:00 with delta_a = 0.117 km
Maneuver detected at 2024-09-05 21:54:05.303223+00:00 with delta_a = 0.142 km
Maneuver detected at 2024-09-29 23:54:56.600372+00:00 with delta_a = 0.154 km
Maneuver detected at 2024-10-01 23: